In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from reconstruct import DiscreteWindowedRampFilter
from geometry import shepp_logan
from project import acquire_projections

In [ ]:
n_pixels = 1001
n_projections = 1200
tau = 0.1

f = shepp_logan(n_pixels)
P = acquire_projections(f, n_projections=n_projections)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 24))

axs[0].imshow(f, cmap="gray")
axs[0].set_title("Shepp-Logan phantom")
axs[0].axis("off")

axs[1].imshow(P, cmap="gray")
axs[1].set_title("Sinogram of Shepp-Logan phantom")
axs[1].axis("off")

fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ax.plot(P[0])
ax.set_xlabel("t")
ax.set_ylabel("Projection")

ax.set_title("Projection at Theta=0º")

In [ ]:
t = np.arange(-n_pixels // 2, n_pixels // 2)

g = DiscreteWindowedRampFilter(tau)
h_t = g(t)

P_0 = P[0]

S_0 = np.fft.fftshift(np.fft.fft(P_0))
H_w = np.fft.fftshift(np.fft.fft(h_t))

w = np.fft.fftshift(np.fft.fftfreq(1001, d=tau))

convolution = np.convolve(P_0, h_t)
t_conv = np.arange(-len(convolution) // 2, len(convolution) // 2)

multiplication = np.fft.ifft(np.fft.ifftshift(S_0 * H_w))

fig, axs = plt.subplots(3, 2, figsize=(24, 36))

axs[0, 0].plot(t, P_0)
axs[0, 0].set_title("Projection at $\\theta$ = 0º")
axs[0, 0].set_xlabel("t")
axs[0, 0].set_ylabel("$P_\\theta (t)$")

axs[0, 1].plot(t, h_t)
axs[0, 1].set_title("Windowed ramp filter")
axs[0, 1].set_xlabel("t")
axs[0, 1].set_ylabel("h(t)")

axs[1, 0].plot(w, np.abs(S_0))
axs[1, 0].set_title("FT (magnitude) of projection at $\\theta$ = 0º")
axs[1, 0].set_xlabel("w")
axs[1, 0].set_ylabel("$S_{\\theta}(w)$")

axs[1, 1].plot(w, np.abs(H_w))
axs[1, 1].set_title("FT (magnitude) of windowed ramp filter")
axs[1, 1].set_xlabel("w")
axs[1, 1].set_ylabel("$H(w)$")

axs[2, 0].plot(t_conv, convolution)
axs[2, 0].set_title("$P_\\theta (t) * h(t)$")
axs[2, 0].set_xlabel("$f(t)$")
axs[2, 0].set_ylabel("$t$")

axs[2, 1].plot(t, multiplication)
axs[2, 1].set_title("$IFT[S_{\\theta} \\times H(w)]$")
axs[2, 1].set_xlabel("$f(t)$")
axs[2, 1].set_ylabel("$t$")

In [ ]:
from scipy import signal

fig, axs = plt.subplots(1, 3, figsize=(36, 12))

axs[0].imshow(f, cmap="gray")
axs[0].set_title("Shepp-Logan phantom")
axs[0].axis("off")

axs[1].imshow(P, cmap="gray")
axs[1].set_title("Sinogram of Shepp-Logan phantom")
axs[1].axis("off")

Q = signal.convolve(P, h_t.reshape(1, -1), mode="same", method="auto")
axs[2].imshow(Q, cmap="gray")
axs[2].set_title("Filtered sinogram of Shepp-Logan phantom")
axs[2].axis("off")

plt.show()

plt.plot(Q[0])

In [ ]:
n_proj, n_det = P.shape
n = np.arange(-n_det // 2, n_det // 2)
t = n * tau

g = DiscreteWindowedRampFilter(tau)
h_t = g(n)

(n_kernel,) = h_t.shape

n_fft = n_det + n_kernel - 1

H = np.fft.rfft(h_t, n_fft)
S = np.fft.rfft(P, n_fft, axis=1)

Q = np.fft.irfft(S * H, n_fft, axis=1)

window_time = np.hamming(n_fft)
W = np.fft.rfft(window_time)

filtered_windowed_sinogram = np.fft.irfft(S * H * W, n_fft, axis=1)

start = (n_kernel - 1) // 2
end = start + n_det
Q = Q[:, start:end]
filtered_windowed_sinogram = filtered_windowed_sinogram[:, start:end]

fig, axs = plt.subplots(2, 2, figsize=(24, 24))

axs[0, 0].imshow(f, cmap="gray")
axs[0, 0].set_title("Shepp-Logan phantom")
axs[0, 0].axis("off")

axs[0, 1].imshow(P, cmap="gray")
axs[0, 1].set_title("Sinogram of Shepp-Logan phantom")
axs[0, 1].axis("off")

axs[1, 0].imshow(Q, cmap="gray")
axs[1, 0].set_title("Filtered sinogram of Shepp-Logan phantom")
axs[1, 0].axis("off")

axs[1, 1].imshow(filtered_windowed_sinogram, cmap="gray")
axs[1, 1].set_title("Filtered windowed sinogram of Shepp-Logan phantom")
axs[1, 1].axis("off")

In [ ]:
from reconstruct import interpolate_projections

t_new, sinogram_new = interpolate_projections(t, P, factor=10)

In [ ]:
fig, axs = plt.subplots(1, 2)

axs[0].imshow(P, cmap="gray")

axs[1].imshow(sinogram_new, cmap="gray")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from geometry import shepp_logan
from project import acquire_projections
from reconstruct import (
    filter_projections,
    interpolate_projections,
    backproject,
    DiscreteWindowedRampFilter,
)

N = 1001
K = [1, 10, 100, 1000]
tau = 0.1
factor = 10

f = shepp_logan(N)

for k in K:
    print(f"Reconstructing with {k} projections")

    # Acquire projections
    P_theta = acquire_projections(f, n_projections=k, axis="x")

    # Filter projections
    Q_theta = filter_projections(P_theta, tau=tau)

    # Interpolate projections
    t = np.arange(-N // 2, N // 2) * tau
    t_new, Q_theta_interp = interpolate_projections(t, Q_theta, factor=factor)

    # Backproject to reconstruct the image
    recon = backproject(Q_theta_interp, N, t_new, tau)

    # Display the reconstruction
    plt.subplot(1, 2, 1)
    plt.imshow(recon, cmap="gray")
    plt.title(f"Reconstruction with {k} projections")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(np.abs(recon - f), cmap="gray")
    plt.title(f"Residual of reconstruction with {k} projections")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
plt.hist(recon.flatten(), label="reconstruction", bins=100)
plt.hist(f.flatten(), label="phantom", bins=100, alpha=0.5)
plt.legend()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from geometry import shepp_logan
from project import acquire_projections
from time import time
from reconstruct import (
    filter_projections,
    interpolate_projections,
    backproject,
    DiscreteWindowedRampFilter,
)

N = 1001
K = [1, 10, 100, 1000]
tau = 0.1
factor = 10

f = shepp_logan(N)

start = time()
P_theta = acquire_projections(f, n_projections=1000, axis="x")
end = time()

print(f"Time taken for acquire_projections: {end - start:.2f} seconds")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from geometry import shepp_logan
from project import acquire_projections
from time import time
from reconstruct import (
    filter_projections,
    interpolate_projections,
    backproject,
    DiscreteWindowedRampFilter,
)

N = 1001
K = [1, 10, 100, 1000]
tau = 0.1
factor = 10

f = shepp_logan(N)

start = time()
P_theta_vectorised = acquire_projections(
    f, n_projections=1000, axis="x", vectorised=True
)
end = time()
print(f"Time taken for acquire_projections_vectorised: {end - start:.2f} seconds")

plt.imshow(P_theta_vectorised, cmap="gray")
plt.show()
# Filter projections
Q_theta = filter_projections(P_theta_vectorised, tau=tau)

# Interpolate projections
t = np.arange(-N // 2, N // 2) * tau
t_new, Q_theta_interp = interpolate_projections(t, Q_theta, factor=10)

# Backproject to reconstruct the image
recon = backproject(Q_theta_interp, N, t_new, tau)

plt.imshow(recon, cmap="gray")

In [ ]:
plt.imshow(np.abs(f - recon), cmap="gray")

In [ ]:
plt.imshow(f, cmap="gray")

In [ ]:
plt.hist(recon.flatten(), label="reconstruction", bins=100)
plt.hist(f.flatten(), label="phantom", bins=100)
plt.legend()

In [ ]:
plt.hist(f.flatten(), label="phantom", bins=100)

In [ ]:
# Filter projections
Q_theta = filter_projections(P_theta_vectorised, tau=tau)

# Interpolate projections
t = np.arange(-N // 2, N // 2) * tau
t_new, Q_theta_interp = interpolate_projections(t, Q_theta, factor=10)

# Backproject to reconstruct the image
recon = backproject(Q_theta_interp, N, t_new, tau)

plt.imshow(recon, cmap="gray")

In [ ]:
recon = (recon - recon.min()) / (recon.max() - recon.min())

plt.imshow(recon, cmap="gray")
plt.show()
plt.imshow(np.abs(recon - f), cmap="gray")
plt.show()

In [ ]:
plt.hist(recon.flatten(), bins=100, label="reconstruction")
plt.hist(f.flatten(), bins=100, label="phantom")
plt.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(12, 48))

axs[0].imshow(f, cmap="gray")

axs[1].imshow(P_theta, cmap="gray")

axs[2].imshow(Q_theta, cmap="gray")

axs[3].imshow(Q_theta_interp, cmap="gray")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from project import acquire_fanbeam_projections
from geometry import shepp_logan, setup_geometry
from reconstruct import (
    interpolate_projections,
    build_filter,
    filter_projections_freq_kernel,
    distance_correction,
    equiangular_backproject,
)


# -----------------------------------
# Set up geometry parameters
# -----------------------------------
N = 256  # phantom size
R_obj = N / 2 * 0.9  # object fits in a 0.9·N radius
D_sd = 1500.0  # source–detector distance in pixels
D_so = 750.0  # source–object distance
fan_angle = 2 * np.arcsin(R_obj / D_so)  # full fan angle covering object
N_det = 512  # detector bins
delta_beta = fan_angle / N_det  # equi‑angular bin spacing
beta = (
    np.arange(N_det) - (N_det - 1) / 2
) * delta_beta  # detector angles wrt central ray
N_views = 1000
C = (0, 0)  # centre of rotation in lab frame
f_interp = 10  # factor by which sinogram is preinterpolated

S, D_theta, D, theta, beta = setup_geometry(N, D_so, D_sd, R_obj, N_det, N_views)

# --------------------------------------------------------
# Generate Shepp-Logan phantom
# --------------------------------------------------------
f = shepp_logan(N)
plt.imshow(f, cmap="gray")
plt.title("Shepp-Logan phantom")
plt.axis("off")
plt.show()

# --------------------------------------------------------
# Acquire fan beam projections of the phantom
# --------------------------------------------------------
P = acquire_fanbeam_projections(f, S, D)
plt.imshow(P, cmap="gray")
plt.title("Sinogram of Shepp-Logan phantom")
plt.axis("off")
plt.show()

# --------------------------------------------------------------
# Reconstruct the Shepp-Logan phantom from fan beam projections
# --------------------------------------------------------------

# Step 1: Factor the sinogram by the source-object distance
P = distance_correction(P, D_so, beta)
plt.imshow(P, cmap="gray")
plt.title("Sinogram of Shepp-Logan phantom (distance corrected)")
plt.axis("off")
plt.show()

# Step 2: Filter the projections using a discrete fan beam filter
g = build_filter(N_det, filter_type="shepp-logan", cutoff=(np.pi / D_sd))
Q = filter_projections_freq_kernel(P, g, delta_beta=delta_beta)
plt.imshow(Q, cmap="gray")
plt.title("Filtered sinogram of Shepp-Logan phantom")
plt.axis("off")
plt.show()

# Step 3: Interpolate projections
beta_interp, Q_interp = interpolate_projections(beta, Q, factor=10)
plt.imshow(Q_interp, cmap="gray")
plt.title("Interpolated filtered sinogram of Shepp-Logan phantom")
plt.axis("off")
plt.show()

# Step 4: Backproject the filtered projections
recon = equiangular_backproject(
    Q_interp, N, D_so, theta, fan_angle, delta_beta, f_interp=f_interp
)
plt.imshow(recon, cmap="gray", origin="lower")
plt.title("Reconstructed image from fan beam projections")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
from src.geometry import setup_parallel_geometry, shepp_logan

D_sd = 400
D_so = 200
N_pixels = 256
N_det = 512
N_views = 300
R_obj = 125

S, D, theta = setup_parallel_geometry(D_so, D_sd, R_obj, N_det, N_views)

image = shepp_logan(N_pixels)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from IPython.display import HTML

# Set up plot
fig, ax = plt.subplots()
scat1 = ax.scatter(S[0, :, 0], S[0, :, 1], color="red", label="Source")
scat2 = ax.scatter(D[0, :, 0], D[0, :, 1], color="blue", label="Detector")

circle = patches.Circle(
    (0, 0), R_obj, fill=False, edgecolor="green", linewidth=2, linestyle="--"
)
ax.add_patch(circle)

beam_patch = patches.Polygon(
    [[0, 0], [0, 0], [0, 0], [0, 0]], closed=True, facecolor="orange", alpha=0.3
)
ax.add_patch(beam_patch)

img_extent = [-N_pixels / 2, N_pixels / 2, -N_pixels / 2, N_pixels / 2]
image[image == 0] = 1

img_artist = ax.imshow(image, cmap="gray", extent=img_extent, zorder=-1)

ax.set_xlim([-300, 300])
ax.set_ylim([-300, 300])
ax.set_aspect("equal")
ax.legend()


def update(frame):
    # Update scatter plots
    scat1.set_offsets(S[frame])
    scat2.set_offsets(D[frame])
    ax.set_title(f"View {frame + 1}/{N_views}")

    edge1 = D[frame][0]
    edge2 = D[frame][-1]

    # Update polygon patch (triangle)
    beam_patch.set_xy([S[frame, 0], edge1, edge2, S[frame, -1]])

    return scat1, scat2


ani = animation.FuncAnimation(
    fig, update, frames=N_views, blit=False, interval=100, repeat=True
)

HTML(ani.to_jshtml())

In [ ]:
from src.project import acquire_parallel_projections

sinogram = acquire_parallel_projections(image, S, D)

plt.imshow(sinogram, cmap="gray")

In [ ]:
plt.scatter(S[0, 0, 0], S[0, 0, 1], label="Source")
plt.scatter(D[0, 0, 0], D[0, 0, 1], label="Detector")

In [ ]:
import matplotlib.pyplot as plt
from src.geometry import shepp_logan, setup_equiangular_geometry

D_sd = 400
D_so = 200
N_pixels = 256
N_det = 512
N_views = 100
R_obj = 125

S, D, theta, beta, fan_angle, delta_beta = setup_equiangular_geometry(
    D_so, D_sd, R_obj, N_det, N_views
)
image = shepp_logan(N_pixels)
image[image == 0] = 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches
from IPython.display import HTML

# Set up plot
fig, ax = plt.subplots()
scat1 = ax.scatter(S[0, 0], S[0, 1], color="red", label="Source")
scat2 = ax.scatter(D[0, :, 0], D[0, :, 1], color="blue", label="Detector")

circle = patches.Circle(
    (0, 0), R_obj, fill=False, edgecolor="green", linewidth=2, linestyle="--"
)
ax.add_patch(circle)

beam_patch = patches.Polygon(
    [[0, 0], [0, 0], [0, 0]], closed=True, facecolor="orange", alpha=0.3
)
ax.add_patch(beam_patch)

img_extent = [-N_pixels / 2, N_pixels / 2, -N_pixels / 2, N_pixels / 2]
img_artist = ax.imshow(image, cmap="gray", extent=img_extent, zorder=-1)

ax.set_xlim([-450, 450])
ax.set_ylim([-450, 450])
ax.set_aspect("equal")
ax.legend()


def update(frame):
    # Update scatter plots
    scat1.set_offsets(S[frame])
    scat2.set_offsets(D[frame])
    ax.set_title(f"View {frame + 1}/{N_views}")

    # Draw lines from source to edges of detector
    edge1 = D[frame][0]
    edge2 = D[frame][-1]

    # Update polygon patch (triangle)
    beam_patch.set_xy([S[frame], edge1, edge2])
    return scat1, scat2


ani = animation.FuncAnimation(
    fig, update, frames=N_views, blit=False, interval=100, repeat=True
)

HTML(ani.to_jshtml())

In [ ]:
plt.imshow(S[0, 0, 0], cmap="gray")
plt.title("Source X coordinate")
plt.colorbar()
plt.show()

plt.imshow(S[..., 1], cmap="gray")
plt.title("Source Y coordinate")
plt.colorbar()
plt.show()

plt.imshow(D[..., 0], cmap="gray")
plt.title("Detector X coordinate")
plt.colorbar()
plt.show()

plt.imshow(D[..., 1], cmap="gray")
plt.title("Detector Y coordinate")
plt.colorbar()
plt.show()